In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import (pprint as pp, pformat as pf)
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
import dmyplant2
dmyplant2.cred()
mp = dmyplant2.MyPlant(3600)

In [2]:
dval = dmyplant2.Validation.load_def_csv("input2.csv")
dval;

In [3]:
i = 0
vale = dval.iloc[i][['serialNumber','Validation Engine','val start']]

In [4]:
e=dmyplant2.Engine.from_sn(mp,vale['serialNumber'], valstart=vale['val start'])
#e=dmyplant2.Engine.from_sn(mp,1486144) # Forsa Hartmoor M1
#e=dmyplant2.Engine.from_sn(mp,1486152) # Forsa Hartmoor M2
#e=dmyplant2.Engine.from_sn(mp,1486176) # Forsa Hartmoor M3
#e=dmyplant2.Engine.from_sn(mp,1486174) # Forsa Hartmoor M4
#e=dmyplant2.Engine.from_sn(mp,1486197) # Forsa Hartmoor M5
#e=dmyplant2.Engine.from_sn(mp,1486198) # Forsa Hartmoor M6
#e=dmyplant2.Engine.from_sn(mp,1486236) # Forsa Hartmoor M7
#e=dmyplant2.Engine.from_sn(mp,1486225) # Forsa Hartmoor M8
#e=dmyplant2.Engine.from_sn(mp,1486251) # Forsa Hartmoor M9
#e=dmyplant2.Engine.from_sn(mp,1486255) # Forsa Hartmoor M10
#e=dmyplant2.Engine.from_sn(mp,1486272) # Forsa Hartmoor M11


In [9]:
for d in e.asset:
    print(d, type(e[d]))
    print(pf(e[d]), '\n')


agentVersion <class 'str'>
'6.8.3' 

associated <class 'list'>
[{'id': 103791, 'model': 'J-Engine', 'serialNumber': '1145166'},
 {'id': 103792, 'model': 'Gen-Set', 'serialNumber': '1145174'}] 

customer <class 'dict'>
{'id': 4294969534, 'name': 'AB IMPIANTI SRL'} 

dataItems <class 'dict'>
{'AGG_AssetAvailability': {'id': 1040109,
                           'name': 'AGG_AssetAvailability',
                           'timestamp': 1548250174976,
                           'unit': '%',
                           'value': 39.0},
 'AGG_AssetPerformanceRate': {'id': 1040110,
                              'name': 'AGG_AssetPerformanceRate',
                              'timestamp': 1548250174976,
                              'unit': '%',
                              'value': 1.0},
 'AGG_AssetReliabilityRate': {'id': 1040111,
                              'name': 'AGG_AssetReliabilityRate',
                              'timestamp': 1548250174976,
                              'unit': '%',


In [ ]:
    if 'Ramp' in d:
        print(d, e[d])

In [ ]:
print(f"loadramp: {e['rP_Ramp_Set']} %/sec => {100.0 / e['rP_Ramp_Set']:4.1f} sec.")
pd.DataFrame.from_dict(e.dash, orient='index').T

In [ ]:
from dfsm import msgFSM
fsm = msgFSM(e, skip_days=7)
fsm.run() # run Finite State Machine

In [ ]:
rdf = pd.DataFrame(fsm._starts)
rda = rdf
#rda = rdf[((rdf['mode'] == 'AUTO') & (rdf['success'] == True))]
#rda = rdf[((rdf['mode'] == 'AUTO') & (rdf['success'] == True) & (rdf['synchronize'] < 300.0) & (rdf['start-preparation'] < 300.0))]
#rda = rdf[((rdf['success'] == True) & (rdf['start-preparation'] < 1000.0))]
rda = rdf[((rdf['success'] == True))]
rda = rda[['success','mode','cumstarttime','start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']]
rda.round(2)

In [ ]:
rdb = rda[rdf['cumstarttime'] > 300.0]
rdb.round(2)

In [ ]:
rdb.describe().round(2).T

In [ ]:
rdc = rda[rdf['load-ramp'] < 120.0]
rdc.round(2)

In [ ]:
rdc.describe().round(2).T

In [ ]:
print("""
****************
** Überblick: **
****************
""")
print(f"{'Motor':9}: {e}")
print(f"{'Meldung':9}- '{'9047 target load reached'}' {'gefunden.' if any(fsm._messages['name'] == '9047') else 'wird berechnet.'}")
print(f"{'Zeitraum':9}: {fsm.first_message:%d.%m.%Y} bis {fsm.last_message:%d.%m.%Y}")
print(f"{'Dauer':9}: {fsm.period.days} Tage")
print()
print(f"{'GESAMT':9}: {rdf.shape[0]:5} Starts, {rdf[rdf.success].shape[0]:5} erfolgreich => {(rdf[rdf.success].shape[0] / rdf.shape[0]) * 100.0:3.1f} % der Starts erfolgreich.")
for mode in ['MANUAL', 'AUTO']:
    print(f"{mode:9}: {rdf[(rdf['mode'] == mode)].shape[0]:5} Starts, {rdf[((rdf.success) & (rdf['mode'] == mode))].shape[0]:5} erfolgreich => {(rdf[((rdf.success) & (rdf['mode'] == mode))].shape[0] / rdf[(rdf['mode'] == mode)].shape[0]) * 100.0:3.1f} % der Starts erfolgreich.")

In [ ]:
rda.describe().round(2).T

In [ ]:
rda[['start-preparation','starter','hochlauf','idle','synchronize','cumstarttime','load-ramp','target-operation']].hist(figsize = (20,12), bins=50, layout=(3,3));
#rda.hist(figsize = (20,12), bins=100, layout=(4,2));

In [ ]:
rda.plot.box(subplots=True, grid = True, figsize=(24,10), layout=(1,7));

In [ ]:
rda.plot.box(subplots=True, grid = True, figsize=(24,10), layout=(1,7), sym='');

In [ ]:
nalarms = []
ct = 0
for i,c in rdf.iterrows():
    if len(c['alarms']) > 0 and not c['success']:
        ct += 1
        print(f"\nStartversuch: {i}, Success: {c['success']}")
        for a in c['alarms']:
            nalarms.append(a['msg'])
            print(f"{c['mode']:15} {a['state']:20} {pd.to_datetime(int(a['msg']['timestamp'])*1e6).strftime('%d.%m.%Y %H:%M:%S')} {a['msg']['name']} {a['msg']['message']}")
print(f"""
***********************************
** {ct:3} nicht erfolgreiche Starts **
***********************************
""")

In [ ]:
al = pd.DataFrame(fsm._pareto(nalarms))
fig = plt.figure();
color = 'purple'

if not al.empty:
    al['msg'] = al['msg'] + ' (' + al['name'] + ')'
    al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title='Alarms in not successful Starts');
    plt.plot();

In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'red'

for phase in phases:
    al = fsm.alarms_pareto(phase)[:20]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
phases = [['start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation'],'start-preparation','starter','hochlauf','idle','synchronize','load-ramp','target-operation']
fig = plt.figure();
color = 'orange'

for phase in phases:
    al = fsm.warnings_pareto(phase)[:20]
    if not al.empty:
        al['msg'] = al['msg'] + ' (' + al['name'] + ')'
        al.set_index('msg').sort_values(by = "anz",ascending=True).plot.barh(y=['anz'],figsize=(16,len(al) / 3.8), color=[color], position = 1.0, grid=True, title=' | '.join(phase) if type(phase) == list else phase);
        plt.plot();


In [ ]:
mfn = './data/' + str(e._sn) + '_messages.txt'
fsm.save_messages('./data/' + str(e._sn) + '_messages.txt')
print(mfn)

In [ ]:
any(fsm._messages['name'] == '9047')

In [ ]:
#fsm.store()